In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import networkx as nx
from community import community_louvain

# Load dataset
df = pd.read_csv('creditcard.csv')

# Separate features and target variable
X = df.drop(['Class'], axis=1)
y = df['Class']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
modularity_scores = []
k_values = range(5, 30, 5)  # Example range for k

for k in k_values:
    # Construct KNN graph
    nn = NearestNeighbors(n_neighbors=k, metric='cosine')
    nn.fit(X_train)
    distances, indices = nn.kneighbors(X_train)
    
    # Build graph
    G = nx.Graph() 
    for i, neighbors in enumerate(indices):
        for j, dist in zip(neighbors, distances[i]):
            if i != j:
                G.add_edge(i, j, weight=1 - dist)
    
    # Apply Louvain clustering
    partition = community_louvain.best_partition(G)
    modularity = community_louvain.modularity(partition, G)
    modularity_scores.append((k, modularity))

# Find k with the highest modularity
optimal_k = max(modularity_scores, key=lambda x: x[1])[0]
print(f"Optimal k based on modularity: {optimal_k}")

# Map partition dictionary to a list aligned with training data indices
train_clusters = np.array([partition[i] for i in range(len(X_train))])

In [ ]:
# Compute cluster prototypes (mean vectors of clusters)
cluster_ids = np.unique(train_clusters)
cluster_prototypes = {}

for cluster_id in cluster_ids:
    cluster_members = X_train[train_clusters == cluster_id]
    cluster_prototype = cluster_members.mean(axis=0)
    cluster_prototypes[cluster_id] = cluster_prototype


In [ ]:
def compute_cluster_distances(X, cluster_prototypes):
    distances = []
    for x in X:
        dists = [np.linalg.norm(x - cluster_prototypes[cluster_id]) for cluster_id in cluster_prototypes]
        distances.append(dists)
    return np.array(distances)

# Compute distances for training data
train_cluster_distances = compute_cluster_distances(X_train, cluster_prototypes)

# Compute distances for test data
test_cluster_distances = compute_cluster_distances(X_test, cluster_prototypes)


In [ ]:
# Combine original features with cluster distance features
X_train_with_features = np.hstack((X_train, train_cluster_distances))
X_test_with_features = np.hstack((X_test, test_cluster_distances))


In [ ]:
# Print shapes of original and new feature sets
print('Original feature set shape:', X_train.shape)
print('New feature set shape:', X_train_with_features.shape)

# Print shapes of original and new test feature sets
print('Original test feature set shape:', X_test.shape)
print('New test feature set shape:', X_test_with_features.shape)
